In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy import integrate

csv_files = ["E:/2024-10-28/SORA_Prompt1_000_ALL.csv", "E:/2024-10-28/SORA_Prompt1_001_ALL.csv", "E:/2024-10-28/SORA_Prompt1_002_ALL.csv", 
             "E:/2024-10-28/SORA_Prompt1_003_ALL.csv", "E:/2024-10-28/SORA_Prompt1_004_ALL.csv", "E:/2024-11-01/SORA_Prompt1_005_ALL.csv", 
             "E:/2024-11-01/SORA_Prompt1_006_ALL.csv", "E:/2024-11-01/SORA_Prompt1_007_ALL.csv", "E:/2024-11-01/SORA_Prompt1_008_ALL.csv", 
             "E:/2024-11-01/SORA_Prompt1_009_ALL.csv", "E:/2024-11-01/SORA_Prompt2_000_ALL.csv", "E:/2024-11-01/SORA_Prompt2_001_ALL.csv",
             "E:/2024-11-01/SORA_Prompt2_002_ALL.csv", "E:/2024-11-12/SORA_Prompt2_004_ALL.csv", "E:/2024-11-12/SORA_Prompt2_005_ALL.csv",
             "E:/2024-11-12/SORA_Prompt11_000_ALL.csv", "E:/2024-11-12/SORA_Prompt11_001_ALL.csv", "E:/2024-11-12/SORA_Prompt11_002_ALL.csv", 
             "E:/2024-11-12/SORA_Prompt11_003_ALL.csv", "E:/2024-11-12/SORA_Prompt11_004_ALL.csv", "E:/2024-11-12/SORA_Long1_000_ALL.csv", 
             "E:/2024-11-12/SORA_Long1_001_ALL.csv", "E:/2024-11-12/SORA_Long1_002_ALL.csv", "E:/2024-11-12/SORA_Long7_000_ALL.csv", 
             "E:/2024-11-12/SORA_Long7_001_ALL.csv", "E:/2024-11-12/SORA_Custom1_000_ALL.csv", "E:/2024-11-12/SORA_Custom1_001_ALL.csv", 
             "E:/2024-11-13/SORA_Custom1_002_ALL.csv",
             "E:/2024-11-12/SORA_Custom2_000_ALL.csv", "E:/2024-11-13/SORA_Custom3_000_ALL.csv", "E:/2024-11-13/SORA_Custom3_001_ALL.csv",
              "E:/2024-11-13/SORA_Custom4_001_ALL.csv", "E:/2024-11-13/SORA_Custom5_000_ALL.csv", 
             "E:/2024-11-13/SORA_Custom5_001_ALL.csv"]

for n in range(len(csv_files)):
    # Replace with the path to your CSV file
    csv_file = csv_files[n]

    # Read the CSV file, skipping the first 15 rows, and use the next row as headers
    df_cleaned = pd.read_csv(csv_file, skiprows=15, header=1, low_memory=False)

    # Ensure the required columns are present
    # Edit the colums_to_extract based on the scope capturing data
    columns_to_extract = ['TIME', 'CH2', 'CH3', 'MATH2']
    if not set(columns_to_extract).issubset(df_cleaned.columns):
        print("One or more of the required columns are missing. Check column names.")
        print(df_cleaned.columns)  # Print available columns for debugging  
    else:
        # Extract the relevant columns and create a copy
        df_extracted = df_cleaned[columns_to_extract].copy()
        #df_extracted = df_extracted[df_extracted.TIME <= 62.75].reset_index(drop=True)
    
    

        # Calculate Pac (AC Power) as CH2 * CH3 (optional)
        # df_extracted['Pac'] = df_extracted['CH2'] * df_extracted['CH3']
        df_extracted['Pac'] = df_extracted['MATH2']

        # Estimate the sampling rate from the TIME column
        time_diff = df_extracted['TIME'].diff().mean()  # Average time difference between samples
        sampling_rate = 1 / time_diff  # Sampling rate in Hz (samples per second)
        #print(f"Estimated sampling rate: {sampling_rate:.2f} Hz")

        # Check for NaN or infinite values in the data
        #print("\nChecking for NaN or infinite values in the data:")
        #print(df_extracted.isnull().sum())
        #print("Infinite values in Pac:", np.isinf(df_extracted['Pac']).sum())

        # Replace infinite values with NaN and drop them
        df_extracted.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_extracted.dropna(inplace=True)

        # Confirm that no NaN or infinite values remain
        #print("\nAfter removing NaN and infinite values:")
        #print(df_extracted.isnull().sum())

        # Proceed only if the DataFrame is not empty after dropping NaNs
        if df_extracted.empty:
            print("\nDataFrame is empty after removing NaN and infinite values. Cannot proceed with calculations.")
        else:
            # Downsample the data to 1000 Hz
            desired_sampling_rate = 1000  # Hz
            downsample_factor = int(sampling_rate / desired_sampling_rate)
            if downsample_factor < 1:
                downsample_factor = 1
            df_extracted = df_extracted.iloc[::downsample_factor, :].reset_index(drop=True)
            sampling_rate = sampling_rate / downsample_factor
            print(f"New sampling rate after downsampling: {sampling_rate:.2f} Hz")

            # Recalculate nyquist and normalized cutoff frequencies
            nyquist = 0.5 * sampling_rate
            cutoff_pdc = 30  # Hz
            normal_cutoff_pdc = cutoff_pdc / nyquist
            center_freq = 120  # Hz
            bandwidth = 10     # Hz
            lowcut = center_freq - bandwidth / 2
            highcut = center_freq + bandwidth / 2
            normal_lowcut = lowcut / nyquist
            normal_highcut = highcut / nyquist

            print(f"Normalized cutoff frequencies for Pdc: {normal_cutoff_pdc}")
            print(f"Normalized cutoff frequencies for P2nd: {normal_lowcut}, {normal_highcut}")


    ##############################################################################################################
            # Design and apply filters if normalized frequencies are valid
            if 0 < normal_cutoff_pdc < 1:
                b_pdc, a_pdc = butter(N=4, Wn=normal_cutoff_pdc, btype='low', analog=False)
                df_extracted['Pdc'] = filtfilt(b_pdc, a_pdc, df_extracted['Pac'])
            else:
                print("Adjusted normal_cutoff_pdc is out of bounds after downsampling.")  
                
            for n in range(len(df_extracted["Pdc"])):
                if df_extracted["Pdc"][n] < 110 or df_extracted["TIME"][n] < 0:
                    df_extracted.drop(index=n,inplace=True) 
                else:
                    break
            df_extracted = df_extracted.reset_index(drop=True)
            for n in range(len(df_extracted["Pdc"])-1,-1,-1):
                if df_extracted["Pdc"][n] < 125:
                    df_extracted.drop(index=n,inplace=True) 
                else:
                    break
            df_extracted = df_extracted.reset_index(drop=True)

            if 0 < normal_lowcut < normal_highcut < 1:
                b_p2nd, a_p2nd = butter(N=4, Wn=[normal_lowcut, normal_highcut], btype='bandpass', analog=False)
                df_extracted['P2nd'] = filtfilt(b_p2nd, a_p2nd, df_extracted['Pac'])
            else:
                print("Adjusted normalized frequencies for P2nd are out of bounds after downsampling.")

            # Check for NaN values in Pdc and P2nd
            nan_count_pdc = df_extracted['Pdc'].isna().sum()
            nan_count_p2nd = df_extracted['P2nd'].isna().sum()
            print(f"Number of NaN values in Pdc: {nan_count_pdc}")
            print(f"Number of NaN values in P2nd: {nan_count_p2nd}")

            # Proceed if no NaN values in Pdc and P2nd
            if nan_count_pdc == 0 and nan_count_p2nd == 0:
                # Calculate Pac_rest (higher-order harmonics)
                df_extracted['Pac_rest'] = df_extracted['Pac'] - df_extracted['Pdc'] - df_extracted['P2nd']

                # Verify that Pdc + P2nd + Pac_rest equals Pac
                verification_diff = (df_extracted['Pac'] - (df_extracted['Pdc'] + df_extracted['P2nd'] + df_extracted['Pac_rest'])).abs().max()
                print(f"Maximum difference between Pac and (Pdc + P2nd + Pac_rest): {verification_diff}")

###############################################################
               ############################################################################################ # Plot the results
                # fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

                # axs[0].plot(df_extracted['TIME'], df_extracted['Pac'], label='Pac (AC Power)', color='blue')
                # axs[0].set_ylabel('Pac')
                # axs[0].legend()

                # axs[1].plot(df_extracted['TIME'], df_extracted['Pdc'], label='Pdc (DC Component)', color='red')
                # axs[1].set_ylabel('Pdc')
                # axs[1].legend()

                # axs[2].plot(df_extracted['TIME'], df_extracted['P2nd'], label='P2nd (2nd Harmonic)', color='green')
                # axs[2].set_ylabel('P2nd')
                # axs[2].legend()

                # axs[3].plot(df_extracted['TIME'], df_extracted['Pac_rest'], label='Pac_rest (Higher-Order Harmonics)', color='purple')
                # axs[3].set_ylabel('Pac_rest')
                # axs[3].legend()

                # axs[3].set_xlabel('Time')

                # plt.suptitle('Power Decomposition: Pac, Pdc, P2nd, and Pac_rest')
                # plt.tight_layout(rect=[0, 0, 1, 0.96])
                # plt.savefig("UnZoomed")
                # plt.show()

            #     ### Zoomed-In Plots ###
            #     # need to adap the time interval you are interested
                
            #     # Zoom-in plot for 2-5 seconds
            #     time_range_1 = (df_extracted['TIME'] >= 40) & (df_extracted['TIME'] <= 43)  #time range subject to change
            #     df_zoom_1 = df_extracted.loc[time_range_1]

            #     fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

            #     axs[0].plot(df_zoom_1['TIME'], df_zoom_1['Pac'], label='Pac (AC Power)', color='blue')
            #     axs[0].set_ylabel('Pac')
            #     axs[0].legend()

            #     axs[1].plot(df_zoom_1['TIME'], df_zoom_1['Pdc'], label='Pdc (DC Component)', color='red')
            #     axs[1].set_ylabel('Pdc')
            #     axs[1].legend()

            #     axs[2].plot(df_zoom_1['TIME'], df_zoom_1['P2nd'], label='P2nd (2nd Harmonic)', color='green')
            #     axs[2].set_ylabel('P2nd')
            #     axs[2].legend()

            #     axs[3].plot(df_zoom_1['TIME'], df_zoom_1['Pac_rest'], label='Pac_rest (Higher-Order Harmonics)', color='purple')
            #     axs[3].set_ylabel('Pac_rest')
            #     axs[3].legend()

            #     axs[3].set_xlabel('Time (40-43s)') #time range subject to change

            #     plt.suptitle('Zoomed-In Power Decomposition: 40-43s')  #time range subject to change
            #     plt.tight_layout(rect=[0, 0, 1, 0.96]) 
            #     plt.savefig("ZoomedIn1")
            #     plt.show()


            #     # Zoom-in plot for 15-17 seconds####################################################################################
            #     time_range_2 = (df_extracted['TIME'] >= 0) & (df_extracted['TIME'] <= 1)  #subject to change
            #     df_zoom_2 = df_extracted.loc[time_range_2]

            #     #fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
            #     fig, axs = plt.subplots(2, 1, figsize=(12, 5), sharex=True)
            #     axs[0].plot(df_zoom_2['TIME'], df_zoom_2['Pac'], label='Pac (AC Power)', color='blue')
            #     axs[0].set_ylabel('AC Power (W)')
            #     axs[0].legend()

            #     axs[1].plot(df_zoom_2['TIME'], df_zoom_2['Pdc'], label='Pdc (DC Component)', color='red')
            #     axs[1].set_ylabel('DC Component (W)')
            #     axs[1].legend()

            #     # axs[2].plot(df_zoom_2['TIME'], df_zoom_2['P2nd'], label='P2nd (2nd Harmonic)', color='green')
            #     # axs[2].set_ylabel('2nd Harmonic (W)')
            #     # axs[2].legend()

            #     # axs[3].plot(df_zoom_2['TIME'], df_zoom_2['Pac_rest'], label='Pac_rest (Higher-Order Harmonics)', color='purple')
            #     # axs[3].set_ylabel('N>2 Harmonics (W)')
            #     # axs[3].legend()

            #     axs[1].set_xlabel('Time (56-56.5s)')  #time range subject to change

            #     plt.suptitle('Zoomed-In Power Decomposition (AC and DC)')  #time range subject to change
            #    # plt.tight_layout(rect=[0, 0, 1, 0.96])
            #     plt.savefig("ZOOMINCROP1_Prompt11_003")
            #     plt.show()
                
            #     # Zoom-in plot for 15-17 seconds####################################################################
            #     time_range_2 = (df_extracted['TIME'] >= 0) & (df_extracted['TIME'] <= df_extracted['TIME'].max())  #subject to change
            #     df_zoom_2 = df_extracted.loc[time_range_2]

            #     #fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
            #     fig, axs = plt.subplots(2, 1, figsize=(12, 5), sharex=True)
            #     axs[0].plot(df_zoom_2['TIME'], df_zoom_2['Pac'], label='Pac (AC Power)', color='blue')
            #     axs[0].set_ylabel('AC Power (W)')
            #     axs[0].legend()

            #     axs[1].plot(df_zoom_2['TIME'], df_zoom_2['Pdc'], label='Pdc (DC Component)', color='red')
            #     axs[1].set_ylabel('DC Component (W)')
            #     axs[1].legend()

            #     # axs[2].plot(df_zoom_2['TIME'], df_zoom_2['P2nd'], label='P2nd (2nd Harmonic)', color='green')
            #     # axs[2].set_ylabel('2nd Harmonic (W)')
            #     # axs[2].legend()

            #     # axs[3].plot(df_zoom_2['TIME'], df_zoom_2['Pac_rest'], label='Pac_rest (Higher-Order Harmonics)', color='purple')
            #     # axs[3].set_ylabel('N>2 Harmonics (W)')
            #     # axs[3].legend()

            #     axs[1].set_xlabel('Time')  #time range subject to change

            #     plt.suptitle('Zoomed-In Power Decomposition (AC and DC)')  #time range subject to change
            #    # plt.tight_layout(rect=[0, 0, 1, 0.96])
            #     plt.savefig("ZOOMINCROP2_Prompt11_003")
            #     plt.show()

            # else:
            #     print("Cannot proceed with plotting due to NaN values in Pdc or P2nd.")

    trap1 = integrate.trapezoid(df_extracted['Pdc'],dx=time_diff)
    trap2 = integrate.cumulative_trapezoid(df_extracted['Pdc'],dx=time_diff)
    simp1 = integrate.simpson(df_extracted['Pdc'],dx=time_diff)
    simp2 = integrate.cumulative_simpson(df_extracted['Pdc'],dx=time_diff)
    totalenergy = (trap1+simp1+trap2[-1]+simp2[-1])/4
    
    print(csv_file + " Power = " + str(totalenergy))

New sampling rate after downsampling: 1041.67 Hz
Normalized cutoff frequencies for Pdc: 0.057600000000000005
Normalized cutoff frequencies for P2nd: 0.22080000000000002, 0.24000000000000005
Number of NaN values in Pdc: 0
Number of NaN values in P2nd: 0
Maximum difference between Pac and (Pdc + P2nd + Pac_rest): 2.2737367544323206e-13
E:/2024-10-28/SORA_Prompt1_000_ALL.csv Power = 1325.5687848634386
New sampling rate after downsampling: 1041.67 Hz
Normalized cutoff frequencies for Pdc: 0.057600000000000005
Normalized cutoff frequencies for P2nd: 0.22080000000000002, 0.24000000000000005
Number of NaN values in Pdc: 0
Number of NaN values in P2nd: 0
Maximum difference between Pac and (Pdc + P2nd + Pac_rest): 2.2737367544323206e-13
E:/2024-10-28/SORA_Prompt1_001_ALL.csv Power = 1415.1663939156908
New sampling rate after downsampling: 1041.67 Hz
Normalized cutoff frequencies for Pdc: 0.0576
Normalized cutoff frequencies for P2nd: 0.2208, 0.24
Number of NaN values in Pdc: 0
Number of NaN val